In [2]:
import argparse
import os
os.umask(0)
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import pickle
import sys
from importlib import import_module

import torch
from torch.utils.data import DataLoader, Sampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from data import ArgoTestDataset
from utils import Logger, load_pretrain

In [3]:
torch.cuda.set_device(2)

In [4]:
# from lanegcn import get_model
# config, Dataset, collate_fn, generator, loss_l2, post_process, opt = get_model()

In [5]:
from lanegcn_style import get_model
config, Dataset, collate_fn, loss_l2, post_process, net0, opt0, net1, opt1, net2, opt2, net3, opt3 = get_model()


In [6]:
# ckpt_path_g = 'results_small_class_style0/lanegcn/generator36.000.ckpt'
# ckpt_g = torch.load(ckpt_path_g, map_location=lambda storage, loc: storage)
# load_pretrain(generator, ckpt_g["state_dict"])

In [1]:
ckpt_path_g = 'results_small_class_style0/lanegcn/generator36.000.ckpt'
ckpt_g = torch.load(ckpt_path_g, map_location=lambda storage, loc: storage)
load_pretrain(net0, ckpt_g["state_dict"])

ckpt_path_g = 'results_small_class_style1/lanegcn/generator36.000.ckpt'
ckpt_g = torch.load(ckpt_path_g, map_location=lambda storage, loc: storage)
load_pretrain(net1, ckpt_g["state_dict"])

ckpt_path_g = 'results_small_class_style2/lanegcn/generator36.000.ckpt'
ckpt_g = torch.load(ckpt_path_g, map_location=lambda storage, loc: storage)
load_pretrain(net2, ckpt_g["state_dict"])

ckpt_path_g = 'results_small_class_style3/lanegcn/generator36.000.ckpt'
ckpt_g = torch.load(ckpt_path_g, map_location=lambda storage, loc: storage)
load_pretrain(net3, ckpt_g["state_dict"])

NameError: name 'torch' is not defined

In [ ]:
generator.eval()
generator.cuda()

Net(
  (actor_net): ActorNet(
    (spatial_embedding): Linear(in_features=3, out_features=64, bias=True)
    (encoder): LSTM(64, 128, batch_first=True)
  )
  (map_net): MapNet(
    (input): Sequential(
      (0): Linear(in_features=2, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (norm): GroupNorm(1, 128, eps=1e-05, affine=True)
        (relu): ReLU(inplace=True)
      )
    )
    (seg): Sequential(
      (0): Linear(in_features=2, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(
        (linear): Linear(in_features=128, out_features=128, bias=False)
        (norm): GroupNorm(1, 128, eps=1e-05, affine=True)
        (relu): ReLU(inplace=True)
      )
    )
    (fuse): ModuleDict(
      (ctr): ModuleList(
        (0): Linear(in_features=128, out_features=128, bias=False)
        (1): Linear(in_features=128, out_features=128, bias=False)
        (2):

In [ ]:
dataset = ArgoTestDataset('val', config, train=False)

In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size=config["val_batch_size"],
    num_workers=config["val_workers"],
    collate_fn=collate_fn,
    shuffle=True,
    pin_memory=True,
)

In [1]:
preds = {}
gts = {}
cities = {}

for ii, data in tqdm(enumerate(data_loader)):
    data = dict(data)
    with torch.no_grad():
        out_rel, output = generator(data)
        results = [x[0:1].detach().cpu().numpy() for x in output["reg"]]
    for i, (argo_idx, pred_traj) in enumerate(zip(data["argo_id"], results)):
        preds[argo_idx] = pred_traj.squeeze()
        cities[argo_idx] = data["city"][i]
        gts[argo_idx] = data["gt_preds"][i][0] if "gt_preds" in data else None
    break


NameError: name 'tqdm' is not defined

In [ ]:
# save for further visualization
res = dict(
    preds = preds,
    gts = gts,
    cities = cities,
)     # res['preds'][9].shape = (6, 30, 2), res['gts']={9:None,30:None...}, res['cities]=(9:'MIA',...,77:'PIT')


torch.save(res,f"{config['save_dir']}/test_JAN9.pkl")
 

In [15]:
from argoverse.evaluation.eval_forecasting import (compute_forecasting_metrics,)
_ = compute_forecasting_metrics(preds, gts, cities, 6, 30, 2)
_ = compute_forecasting_metrics(preds, gts, cities, 1, 30, 2)

------------------------------------------------
Prediction Horizon : 30, Max #guesses (K): 6
------------------------------------------------
{'minADE': 4.504263159282657, 'minFDE': 9.362756201710523, 'MR': 0.91875, 'DAC': 0.8475694444444437}
------------------------------------------------
------------------------------------------------
Prediction Horizon : 30, Max #guesses (K): 1
------------------------------------------------
{'minADE': 15.123947759621716, 'minFDE': 28.750673909532818, 'MR': 0.9958333333333333, 'DAC': 0.99375}
------------------------------------------------
